<a href="https://colab.research.google.com/github/kcj4800/Tensorflow_practice/blob/main/%EC%87%BC%ED%95%91%EB%AA%B0_%EC%82%AC%EC%A7%84_%EA%B5%AC%EB%B6%84_%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np


(trainX, trainY), (testX, testY) = tf.keras.datasets.fashion_mnist.load_data()

trainX = trainX / 255.0
testX = testX / 255.0

trainX = trainX.reshape((trainX.shape[0],trainX.shape[1], trainX.shape[2], 1))
testX = testX.reshape((testX.shape[0], testX.shape[1], testX.shape[2], 1))

# sequential : 순차적인, 결과로서 일어나는
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax'),
])

# model.summary()를 해주고싶으면 첫번째 레이어에 input_shape(28, 28, 1)등을 꼭 해줘야한다.
# Conv2D나 MaxPooling2D등이 없을 경우 Flatten(input_shape=(28, 28, 1))등으로 쓰면 된다.
model.summary()

# sparse : 드문 드문, 희박한, 성긴
# sparse categorical crossentropy : 희소 범주형 고차 엔트로피, 
# 엔트로피: 어떤 계통 안에서 일어나는 일의 빈도를 나타내는 척도(단위)
# compile : 편집하다.
# optimizer : 최적화.
# metrics : 측정의 기준
# accuracy : 정확성, 정밀도
# validation : 확인, (동) 입증하다, 허가하다

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(trainX, trainY, validation_data=(testX, testY), epochs=3)




4422102/4422102 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                           

In [ ]:
model.save('새폴더/model1')

In [ ]:
불러온모델 = tf.keras.models.load_model('새폴더/model1')
불러온모델.summary()

# tensorflow 2.3v 이하에서 발생하는 버그 발생시
# 불러온모델.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics='sparse_categorical_accuracy')
# 이 후에 model.evaluate(), model.predict(), model.fit() 등을 하면 된다.

불러온모델.evaluate(testX, testY)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 dense (Dense)               (None, 64)                2

[0.2499871402978897, 0.9090999960899353]

In [ ]:
# 2. w값만 저장/ 로드 => 모든 w값은 variables 폴더 안에 variables.data, variables.index 등으로 존재
# check point 저장 => epochs를 돌리는 중간 중간 체크포인트 저장이 가능하다.

import tensorflow as tf
import numpy as np


(trainX, trainY), (testX, testY) = tf.keras.datasets.fashion_mnist.load_data()

trainX = trainX / 255.0
testX = testX / 255.0

trainX = trainX.reshape((trainX.shape[0],trainX.shape[1], trainX.shape[2], 1))
testX = testX.reshape((testX.shape[0], testX.shape[1], testX.shape[2], 1))

# sequential : 순차적인, 결과로서 일어나는
model = tf.keras.Sequential([
    # tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(28,28,1)),
    # tf.keras.layers.MaxPooling2D((2, 2)),
    # tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    # tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax'),
])
# 콜백함수를 선언했으면, model.fit() 할때 callbacks=[] 함수를 집어넣어주면 하나의 epoch마다 w값을 저장해준다.
콜백함수 = tf.keras.callbacks.ModelCheckpoint(
    filepath='체크포인트/mnist', # 체크포인트 폴더명 안에 mnist라는 파일이 생성됨(덮어쓰기로 저장)
    # filepath='체크포인트/mnist{epoch}', # 체크포인트를 매 epoch 마다 저장해줌
    # validation_accuracy가 최대가 되는 check point만 저장하고 싶을 때
    monitor='val_acc',
    mode='max',
    # 위의 두가지 함수를 이용하면 되지만, 이 경우 val_acc값이 뻥튀기 될 때가 있어, 악용될 수 있음에 유의해야한다.
    save_weights_only=True,
    save_freq='epoch'  
  )



# model.summary()를 해주고싶으면 첫번째 레이어에 input_shape(28, 28, 1)등을 꼭 해줘야한다.
# Conv2D나 MaxPooling2D등이 없을 경우 Flatten(input_shape=(28, 28, 1))등으로 쓰면 된다.
model.summary()

# sparse : 드문 드문, 희박한, 성긴
# sparse categorical crossentropy : 희소 범주형 고차 엔트로피, 
# 엔트로피: 어떤 계통 안에서 일어나는 일의 빈도를 나타내는 척도(단위)
# compile : 편집하다.
# optimizer : 최적화.
# metrics : 측정의 기준
# accuracy : 정확성, 정밀도
# validation : 확인, (동) 입증하다, 허가하다

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(trainX, trainY, validation_data=(testX, testY), epochs=3, callbacks=[콜백함수])





Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_11 (Flatten)        (None, 784)               0         
                                                                 
 dense_26 (Dense)            (None, 128)               100480    
                                                                 
 dense_27 (Dense)            (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
1875/1875 [==============================] - 7s 3ms/step - loss: 0.4939 - accuracy: 0.8269 - val_loss: 0.4220 - val_accuracy: 0.8488
Epoch 2/3
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3709 - accuracy: 0.8662 - val_loss: 0.3918 - val_accuracy: 0.8611
Epoch 3/3
1875/1875 [====

In [ ]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax'),
])

model2.summary()

model2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model2.load_weights('체크포인트/mnist')
model2.fit(trainX, trainY, validation_data=(testX, testY), epochs=3)
model2.evaluate(testX, testY)

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_14 (Flatten)        (None, 784)               0         
                                                                 
 dense_32 (Dense)            (None, 128)               100480    
                                                                 
 dense_33 (Dense)            (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


Epoch 1/3
1875/1875 [==============================] - 13s 6ms/step - loss: 0.3098 - accuracy: 0.8862 - val_loss: 0.3564 - val_accuracy: 0.8721
Epoch 2/3
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2954 - accuracy: 0.8922 - val_loss: 0.3431 - val_accuracy: 0.8785
Epoch 3/3
313/313 [==============================] - 1s 2ms/step - loss: 0.3511 - accuracy: 0.8762


[0.3511282205581665, 0.8762000203132629]